In [2]:
# 📦 1. IMPORTS AND ENVIRONMENT SETUP
from zumi.zumi import Zumi
from zumi.util.screen import Screen
from datetime import datetime
from zumi.util.vision import Vision
from zumi.util.camera import Camera 
from zumi.personality import Personality
import time
import pandas as pd
from collections import defaultdict
import cv2

zumi =    Zumi()
camera    = Camera()
screen    = Screen()
vision    = Vision()
personality = Personality(zumi, screen)

In [ ]:
# ➡️ 2. EVENT LOGGER CLASS

class EventLogger:
    """
    Records events (action names) with timestamps in memory.
    On request, exports data to a CSV file.

    Attributes:
        log (defaultdict[list]): maps action (str) → list of datetime timestamps.
    """

    def __init__(self):
        self.log = defaultdict(list)

    def log_event(self, action: str):
        """
        Add the event 'action' with the current timestamp to the internal log.
        """
        self.log[action].append(datetime.now())

    def save_to_csv(self, output_folder: str = "."):
        """
        Save all logged events into a CSV file under `output_folder`.
        Filename format: “zumi_events_<YYYYMMDD_HHMMSS>.csv”.
        """
        rows = []
        for action, timestamps in self.log.items():
            for ts in timestamps:
                rows.append({"timestamp": ts, "action": action})

        df = pd.DataFrame(rows)
        if not df.empty:
            df = df.sort_values(by="timestamp")
            current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
            file_name = os.path.join(output_folder, f"zumi_events_{current_time}.csv")
            df.to_csv(file_name, index=False)
            print(f"Events saved to {file_name}")
        else:
            print("No events to save.")

In [ ]:
# 🤖 3. ZUMI DRIVER CLASS

class ZumiDriver:
    """
    Controls the Zumi robot’s movement, line-following, and turning operations.

    Dependencies:
        - zumi: object providing low-level Zumi API (movement, sensors, gyro).
        - camera: object for camera control (capture, show_image, close).
        - vision: object for computer vision tasks (QR, face detection).
        - screen: object for drawing text/images on Zumi’s screen.
        - personality: object for setting Zumi’s mood (happy, angry, celebrate).
        - logger: instance of EventLogger to record key events.
    """

    def __init__(self, zumi, camera, vision, screen, personality, logger):
        self.zumi = zumi
        self.camera = camera
        self.vision = vision
        self.screen = screen
        self.personality = personality
        self.logger = logger

    def read_ir(self):
        """
        Read IR sensor values from Zumi.
        Returns:
            Tuple[int, int, int, int, int, int]:
            (front_right, bottom_right, back_right, bottom_left, back_left, front_left)
        """
        return self.zumi.get_all_IR_data()

    def line_correction(self, bottom_left, bottom_right, desired_angle, threshold):
        """
        Adjust desired_angle based on bottom IR sensors:
        - If bottom_left > threshold and bottom_right < threshold: desired_angle += 5
        - If bottom_left < threshold and bottom_right > threshold: desired_angle -= 5
        In both cases, stop for a brief moment for stability.
        """
        if bottom_left > threshold and bottom_right < threshold:
            desired_angle += 5
            self.zumi.stop()
            time.sleep(0.01)
        elif bottom_left < threshold and bottom_right > threshold:
            desired_angle -= 5
            self.zumi.stop()
            time.sleep(0.01)
        return desired_angle

    def turning_correction(self, desired_angle, turn_angle):
        """
        Simplify correction: compute delta = |turn_angle - |desired_angle||,
        then restore sign: return -delta if desired_angle < 0 else +delta.
        """
        delta = abs(turn_angle - abs(desired_angle))
        return -delta if desired_angle < 0 else delta

    def turn_90(self, direction):
        """
        Turn Zumi exactly 90 degrees in `direction` ('left' or 'right'):
        - Reset gyro, signal turn, execute 90°, stop signal, log event.
        """
        self.zumi.reset_gyro()
        if direction == "left":
            self.zumi.signal_left_on()
            self.zumi.turn_left(90)
            self.zumi.signal_left_off()
            self.logger.log_event("move_left")
        elif direction == "right":
            self.zumi.signal_right_on()
            self.zumi.turn_right(90)
            self.zumi.signal_right_off()
            self.logger.log_event("move_right")

    def turn_to_check(self, direction, angle=90):
        """
        Partially turn to check for a line:
        - If direction == 'left', turn left by `angle`.
        - If direction == 'right', turn right by `2 * angle`.
        Then return the gyro reading.
        """
        self.zumi.reset_gyro()
        if direction == "left":
            self.zumi.signal_left_on()
            self.zumi.turn_left(angle)
            self.zumi.signal_left_off()
        else:
            self.zumi.signal_right_on()
            self.zumi.turn_right(2 * angle)
            self.zumi.signal_right_off()
        time.sleep(0.01)
        return self.zumi.read_z_angle()

    def move_after_turn(self, speed, desired_angle, times=3):
        """
        After realigning heading, move straight for `times` iterations,
        then call stop().
        """
        self.zumi.reset_gyro()
        for _ in range(times):
            self.zumi.go_straight(speed, desired_angle)
        self.zumi.stop()

    def approach_line_front(self, speed, threshold):
        """
        Move forward until both bottom sensors detect the line (values < threshold).
        """
        while True:
            _, bottom_right, _, bottom_left, _, _ = self.read_ir()
            self.zumi.go_straight(speed, 0)
            if bottom_right < threshold and bottom_left < threshold:
                self.zumi.stop()
                break

    def follow_line_until_loss(self, speed, initial_angle, threshold):
        """
        Follow the line while either bottom sensor is on-line (value < threshold).
        When both bottom sensors > threshold → exit and stop().
        """
        desired_angle = initial_angle
        while True:
            _, br, _, bl, _, _ = self.read_ir()
            if br > threshold and bl > threshold:
                self.zumi.stop()
                break
            desired_angle = self.turning_correction(desired_angle, 90)
            self.zumi.go_straight(speed, desired_angle)

    def circle(self, direction, number_of_objects, threshold, speed=4):
        """
        Drive a square 'circle' around each detected line segment:
        1. Approach the line from front.
        2. For each object, do 4 разів: turn_90 + follow_line_until_loss.
        """
        self.logger.log_event(f"start_{direction}_circle")
        self.zumi.reset_gyro()

        # Step 1: get on the line frontally
        self.approach_line_front(speed, threshold)

        # Step 2: for each object, perform 4 сегменти квадрата
        for _ in range(number_of_objects):
            for _ in range(4):
                self.turn_90(direction)
                desired_angle = self.zumi.read_z_angle()
                desired_angle = self.turning_correction(desired_angle, 90)
                self.follow_line_until_loss(speed, desired_angle, threshold)

        self.logger.log_event(f"done_{direction}_circle")

    def find_finish_line(self):
        """
        Detect final finish line and realign:
        - Try turn left (80°). If no line, try turn right (80°).
        - Once on finish line, drive forward until both bottom sensors > threshold.
        - Then move_after_turn and final 25° turn.
        """
        speed = 3
        threshold = 100
        self.zumi.reset_gyro()

        # Try turn left side
        turned_left_angle = self.turn_to_check("left", 80)
        _, br, _, bl, _, _ = self.read_ir()

        if bl > threshold and br > threshold:
            # Found line on left
            print("line left")
            while True:
                self.zumi.turn_right(5)
                _, br, _, bl, _, _ = self.read_ir()
                if not (bl > threshold and br > threshold):
                    break
            self.zumi.reset_gyro()
            while True:
                _, br, _, bl, _, _ = self.read_ir()
                self.zumi.go_straight(speed, 0)
                if br > threshold and bl > threshold:
                    self.zumi.stop()
                    break
            self.move_after_turn(speed, 0, 15)
            self.zumi.turn_right(25)

        else:
            # No line on left → try right
            print("no line left")
            turned_right_angle = self.turn_to_check("right", 80)
            _, br, _, bl, _, _ = self.read_ir()

            while bl > threshold and br > threshold:
                self.zumi.turn_left(5)
                _, br, _, bl, _, _ = self.read_ir()
            self.zumi.reset_gyro()
            while True:
                _, br, _, bl, _, _ = self.read_ir()
                self.zumi.go_straight(speed, 0)
                if br > threshold and bl > threshold:
                    print("line found")
                    self.zumi.stop()
                    break
            self.move_after_turn(speed, 0, 15)
            self.zumi.turn_left(25)

        self.zumi.reset_gyro()

    def find_line(self, angle_to_turn):
        """
        After a 360° dance, realign with the line:
        1. Move until both bottom sensors < threshold (off-line).
        2. Move until both bottom sensors > threshold (back on-line).
        3. If bounce < 0.3 s → correct path and go to finish.
        4. Else if bounce < 1 s → go to finish.
        """
        self.zumi.reset_gyro()
        speed = 5
        threshold = 100

        # 1) Move until off the line
        while True:
            _, br, _, bl, _, _ = self.read_ir()
            self.zumi.go_straight(speed, angle_to_turn)
            if br < threshold and bl < threshold:
                self.zumi.stop()
                break

        # 2) Move until back on the line
        while True:
            _, br, _, bl, _, _ = self.read_ir()
            self.zumi.go_straight(speed, angle_to_turn)
            if br > threshold and bl > threshold:
                self.zumi.stop()
                break

        self.logger.log_event("again_on_line")
        start_time = time.time()

        # 3) Up to 1 s to see if we immediately go off→on again
        while True:
            if time.time() - start_time > 1:
                self.zumi.stop()
                print("time is out")
                return
            _, br, _, bl, _, _ = self.read_ir()
            self.zumi.go_straight(speed, angle_to_turn)
            if br < threshold and bl < threshold:
                self.zumi.stop()
                break

        self.logger.log_event("out_of_line")
        dt = (self.logger.log["out_of_line"][-1] - self.logger.log["again_on_line"][-1]).total_seconds()

        if dt < 0.3:
            # Bounce < 0.3 s → correct then go to finish
            while True:
                _, br, _, bl, _, _ = self.read_ir()
                self.zumi.go_straight(speed, angle_to_turn)
                if br > threshold and bl > threshold:
                    self.zumi.stop()
                    break
            while True:
                _, br, _, bl, _, _ = self.read_ir()
                self.zumi.go_straight(speed, angle_to_turn)
                if br < threshold and bl < threshold:
                    self.zumi.stop()
                    break
            self.move_after_turn(speed, 0, 3)
            self.find_finish_line()

        elif dt < 1:
            # Bounce < 1 s → одразу до finish
            print("find_line_2")
            self.move_after_turn(speed, 0, 3)
            self.find_finish_line()

    def dance_360(self, direction, times):
        """
        Perform a 'dance' by spinning 360° in the specified direction, `times` times.
        Returns the final remainder angle.
        """
        result_angle = 0
        for _ in range(times):
            self.zumi.reset_gyro()
            if direction == "left":
                self.zumi.signal_left_on()
                angle_to_turn = 360
                while angle_to_turn > 10:
                    self.zumi.turn_left(angle_to_turn)
                    angle_to_turn = 360 - self.zumi.read_z_angle()
                self.zumi.signal_left_off()
                self.zumi.stop()
                result_angle = angle_to_turn
            else:
                self.zumi.signal_right_on()
                angle_to_turn = 360
                while angle_to_turn > 10:
                    self.zumi.turn_right(angle_to_turn)
                    angle_to_turn = 360 + self.zumi.read_z_angle()
                self.zumi.signal_right_off()
                self.zumi.stop()
                result_angle = angle_to_turn
        return result_angle

    def finish_with_180(self):
        """
        Perform a final 180° turn and log "finish" event.
        """
        self.zumi.stop()
        self.logger.log_event("finish")
        print("Reached end. Performing 180° turn.")
        self.screen.draw_text_center("Finisher box\nTurning 180°")
        self.zumi.turn_left(180)
        self.screen.draw_text_center("Done!")

    def go_straight(self, speed, desired_angle):
        """
        Wrapper around the low-level zumi.go_straight() call.
        """
        self.zumi.go_straight(speed, desired_angle)

    def run(self, qr_handler):
        """
        Main loop:
        1. Drive forward until detecting an object (front IR sensors).
        2. If object found, delegate to qr_handler.process_object().
        3. If a face is detected, exit loop and finish.
        4. On exit: perform 180° turn, save logs.
        """
        threshold = 100
        speed_forward = 4
        number_of_faces = 0
        number_of_objects = 0

        while True:
            front_right, _, _, _, _, front_left = self.read_ir()
            if front_right < threshold and front_left < threshold:
                number_of_objects += 1
                number_of_faces = qr_handler.process_object(
                    speed=speed_forward,
                    number_of_objects=number_of_objects,
                    number_of_faces=number_of_faces,
                    threshold=threshold,
                )
                if number_of_faces > 0:
                    break
            else:
                self.go_straight(speed_forward, 0)

        self.finish_with_180()
        self.logger.save_to_csv()

In [ ]:
# 📷 4. QRCodeFaceHandler CLASS

class QRCodeFaceHandler:
    """
    Handles detection of QR codes or faces using the camera,
    and triggers appropriate ZumiDriver methods for each recognized command.
    """

    def __init__(self, driver, logger):
        """
        Args:
            driver (ZumiDriver): instance of ZumiDriver
            logger (EventLogger): instance of EventLogger
        """
        self.driver = driver
        self.logger = logger

        # Dispatcher dict for QR messages → handler methods
        self.QR_HANDLERS = {
            "Left Circle": self._cmd_left_circle,
            "Right Circle": self._cmd_right_circle,
            "Turn Left": self._cmd_turn_left,
            "Turn Right": self._cmd_turn_right,
            "Stop": self._cmd_stop,
            "Zumi is happy today!": self._cmd_happy,
            "Zumi is angry today!": self._cmd_angry,
            "Zumi is celebrating today!": self._cmd_celebrate,
        }

    def _cmd_left_circle(self, args: dict):
        """
        Handler for "Left Circle" QR command.
        """
        number_of_objects = args.get("number_of_objects", 1)
        threshold = args.get("threshold", 100)
        speed = args.get("speed", 4)
        self.driver.circle("left", number_of_objects, threshold, speed)

    def _cmd_right_circle(self, args: dict):
        """
        Handler for "Right Circle" QR command.
        """
        number_of_objects = args.get("number_of_objects", 1)
        threshold = args.get("threshold", 100)
        speed = args.get("speed", 4)
        self.driver.circle("right", number_of_objects, threshold, speed)

    def _cmd_turn_left(self, args: dict):
        """
        Handler for "Turn Left" QR command.
        """
        self.driver.zumi.signal_left_on()
        self.driver.zumi.turn_left(90)
        self.driver.zumi.signal_left_off()

    def _cmd_turn_right(self, args: dict):
        """
        Handler for "Turn Right" QR command.
        """
        self.driver.zumi.signal_right_on()
        self.driver.zumi.turn_right(90)
        self.driver.zumi.signal_right_off()

    def _cmd_stop(self, args: dict):
        """
        Handler for "Stop" QR command.
        """
        self.driver.zumi.stop()

    def _cmd_happy(self, args: dict):
        """
        Handler for "Zumi is happy today!" QR command.
        """
        self.driver.personality.happy()

    def _cmd_angry(self, args: dict):
        """
        Handler for "Zumi is angry today!" QR command.
        """
        self.driver.personality.angry()

    def _cmd_celebrate(self, args: dict):
        """
        Handler for "Zumi is celebrating today!" QR command.
        """
        self.driver.personality.celebrate()

    def _handle_360_message(self, message: str):
        """
        Parse and handle 360° spin messages, e.g., "3 times spin 360 left,happy".
        Format: "<n> times spin 360 <direction>,<emotion>"
        """
        parts = message.replace(",", " ").split()
        # Example: ["3", "times", "spin", "360", "left", "happy"]
        try:
            times = int(parts[0])
            direction = parts[4]
            emotion = parts[5]
        except (IndexError, ValueError):
            print("Invalid 360 message format")
            return

        angle_to_turn = self.driver.dance_360(direction, times) * 2

        if emotion == "happy":
            self.driver.personality.happy()
        elif emotion == "angry":
            self.driver.personality.angry()
        elif emotion == "celebrating":
            self.driver.personality.celebrate()

        self.driver.zumi.stop()
        print(f"Angle to turn: {angle_to_turn}")
        self.driver.find_line(angle_to_turn)

    def process_object(self, speed, number_of_objects, number_of_faces, threshold):
        """
        Called when an object is detected.
        Attempts up to 5 frames to find a QR code or a face.
        Returns the updated number_of_faces (incremented if face found).
        """
        self.driver.camera.start_camera()

        for _ in range(5):
            frame = self.driver.camera.capture()
            self.driver.camera.show_image(frame)

            qr_message = self._check_qr_code(frame)
            if qr_message is not None:
                self.driver.camera.close()
                self.driver.screen.draw_text_center("QR Code Detected!")
                self.logger.log_event(f"qr_code_command:{qr_message}")
                print("QR code detected!")

                handler = self.QR_HANDLERS.get(qr_message)
                if handler:
                    handler(
                        {
                            "speed": speed,
                            "number_of_objects": number_of_objects,
                            "threshold": threshold,
                        }
                    )
                    self.logger.log_event(f"qr_code_command:{qr_message}_done")
                else:
                    if "360" in str(qr_message):
                        self._handle_360_message(qr_message)
                    else:
                        print("Invalid QR command")
                break  # stop after handling QR

            # If no QR, convert to gray and check face
            gray = self.driver.vision.convert_to_gray(frame)
            if self._check_face(gray):
                self.driver.camera.close()
                self.driver.screen.draw_text_center("Face Detected!")
                self.logger.log_event("face_detected")
                print("Face detected!")
                timestamp = time.strftime("%Y%m%d_%H%M%S")
                filename = f"{timestamp}.png"
                cv2.imwrite(filename, gray)
                number_of_faces += 1
                break

            time.sleep(1)

        self.driver.camera.close()
        return number_of_faces

    def _check_qr_code(self, frame):
        """
        Try to detect and decode a QR code in the given frame.
        Returns:
            Decoded message (str) or None if not found.
        """
        qr_code = self.driver.vision.find_QR_code(frame)
        if qr_code is not None:
            return self.driver.vision.get_QR_message(qr_code)
        return None

    def _check_face(self, gray_frame):
        """
        Detect a face in a grayscale image.
        Returns True if found, else False.
        """
        return self.driver.vision.find_face(gray_frame) is not None

In [ ]:
# 🏃 5. MAIN LOOP (RUN)

# Instantiate mocks or real APIs
# For demonstration, використаємо «Mock» обʼєкти,
# але в реальному середовищі замініть їх на справжні з Zumi SDK.


# Ініціалізуємо драйвер і обробника QR/face
driver = ZumiDriver(
    zumi=zumi,
    camera=camera,
    vision=vision,
    screen=screen,
    personality=personality,
    logger=logger,
)
qr_handler = QRCodeFaceHandler(driver=driver, logger=logger)

# Запускаємо основний цикл (цю клітинку можна зупинити вручну, наприклад, клавішею «Stop» у Jupyter)
driver.run(qr_handler)

In [3]:
zumi.get_battery_percent()

In [4]:
zumi.mpu.calibrate_MPU()

In [39]:
log = {}
def line_correction(bottom_left, bottom_right, desired_angle, threshold):
    if bottom_left > threshold and bottom_right < threshold:
        desired_angle +=5
        zumi.stop()
        time.sleep(0.01) 
    elif bottom_left < threshold and bottom_right > threshold:
        desired_angle -=5
        zumi.stop()
        time.sleep(0.01) 
    return desired_angle

def turning_correction(desired_angle, turn_angle):
    delta = abs(turn_angle - abs(desired_angle))
    return -delta if desired_angle < 0 else delta

def turn_to_check(turn, angle = 90):
    zumi.reset_gyro()
    if turn == 'left':
        zumi.signal_left_on()
        zumi.turn_left(angle)
        zumi.signal_left_off()
    elif turn == 'right':
        zumi.signal_right_on()
        zumi.turn_right(2*angle)
        zumi.signal_right_off()
    time.sleep(0.01)
    desired_angle = zumi.read_z_angle()
    return desired_angle

def move_after_turning(speed, desired_angle, times=3):
    zumi.reset_gyro() 
    for x in range(times):
        zumi.go_straight(speed, desired_angle)
    zumi.stop()

def object_detected(threshold=100):
    front_right, bottom_right, back_right, bottom_left, back_left, front_left = zumi.get_all_IR_data() # Get center IR sensor value
    
    return front_right < threshold and front_left < threshold



def turn_90_degrees(turn):
    zumi.reset_gyro()
    if turn == 'left':
        zumi.signal_left_on()
        zumi.turn_left(90)
        zumi.signal_left_off()
        log_event('move_left')
    elif turn == 'right':
        zumi.signal_right_on()
        zumi.turn_right(90)
        zumi.signal_right_off()
        log_event('move_right')

def circle(turn, number_of_objects, threshold, speed=4):
    log_event("start "+ turn+ " circle")
    zumi.reset_gyro()
    while True:
        front_right, bottom_right, back_right, bottom_left, back_left, front_left = zumi.get_all_IR_data()
        zumi.go_straight(speed, 0)
        if bottom_right < threshold and bottom_left < threshold:
            zumi.stop()
            break
    for j in range(number_of_objects):
        for i in range(4):
            zumi.reset_gyro()
            turn_90_degrees(turn)
            desired_angle = zumi.read_z_angle()
            desired_angle = turning_correction(desired_angle, 90)
            front_right, bottom_right, back_right, bottom_left, back_left, front_left = zumi.get_all_IR_data()
            zumi.reset_gyro()
            while bottom_left > threshold or bottom_right > threshold:
                front_right, bottom_right, back_right, bottom_left, back_left, front_left = zumi.get_all_IR_data()
                desired_angle = line_correction(bottom_left, bottom_right, desired_angle, threshold)
                zumi.go_straight(speed, desired_angle)
            zumi.stop()
    log_event("done "+ turn+ " circle")

def find_the_finish_line():
    speed = 3
    threshold = 100
    zumi.reset_gyro()

    turned_left_angle = turn_to_check('left', 80)
    front_right, bottom_right, back_right, bottom_left, back_left, front_left = zumi.get_all_IR_data()
    if bottom_left > threshold and bottom_right > threshold:
        print("line left")
        while (bottom_left > threshold and bottom_right > threshold):
            zumi.turn_right(5)
            front_right, bottom_right, back_right, bottom_left, back_left, front_left = zumi.get_all_IR_data()
        
        zumi.reset_gyro()
        while True:
            front_right, bottom_right, back_right, bottom_left, back_left, front_left = zumi.get_all_IR_data()
            zumi.go_straight(speed, 0)
            if bottom_right > threshold and bottom_left > threshold:
                zumi.stop()
                break
        
        move_after_turning(speed, 0, 15)
        zumi.turn_right(25)
    else:
        print("no line left")
        turned_right_angle = turn_to_check('right', 80)        
        front_right, bottom_right, back_right, bottom_left, back_left, front_left = zumi.get_all_IR_data()
        
        while (bottom_left > threshold and bottom_right > threshold):
            zumi.turn_left(5)
            front_right, bottom_right, back_right, bottom_left, back_left, front_left = zumi.get_all_IR_data()
        
        zumi.reset_gyro()
        while True:
            front_right, bottom_right, back_right, bottom_left, back_left, front_left = zumi.get_all_IR_data()
            zumi.go_straight(speed, 0)
            if bottom_right > threshold and bottom_left > threshold:
                print("line found")
                zumi.stop()
                break
        
        move_after_turning(speed, 0, 15)
        
        zumi.turn_left(25)
    zumi.reset_gyro()
    
def find_line(angle_to_turn):
    zumi.reset_gyro()
    speed = 5
    threshold = 100
    # Move if sitll on line
    
    while True:
        front_right, bottom_right, back_right, bottom_left, back_left, front_left = zumi.get_all_IR_data()
        zumi.go_straight(speed, angle_to_turn)
        if bottom_right < threshold and bottom_left < threshold:
            zumi.stop()
            break
    # Move to find new line
    while True:
        front_right, bottom_right, back_right, bottom_left, back_left, front_left = zumi.get_all_IR_data()
        zumi.go_straight(speed, angle_to_turn)
        if bottom_right > threshold and bottom_left > threshold:
            zumi.stop()
            break
    log_event("again_on_line")

    start_time = time.time()
    while True:
        if time.time() - start_time > 1:
            zumi.stop()
            print("time is out")
            break
        front_right, bottom_right, back_right, bottom_left, back_left, front_left = zumi.get_all_IR_data()
        zumi.go_straight(speed, angle_to_turn)
        if bottom_right < threshold and bottom_left < threshold:
            zumi.stop()
            break

    log_event("out_of_line")
    if (log['out_of_line'][-1] - log['again_on_line'][-1]).total_seconds() < 0.3 :
        while True:
            front_right, bottom_right, back_right, bottom_left, back_left, front_left = zumi.get_all_IR_data()
            zumi.go_straight(speed, angle_to_turn)
            if bottom_right > threshold and bottom_left > threshold:
                zumi.stop()
                break
        while True:
            front_right, bottom_right, back_right, bottom_left, back_left, front_left = zumi.get_all_IR_data()
            zumi.go_straight(speed, angle_to_turn)
            if bottom_right < threshold and bottom_left < threshold:
                zumi.stop()
                break

        move_after_turning(speed, 0, 3)
        find_the_finish_line()
    elif (log['out_of_line'][-1] - log['again_on_line'][-1]).total_seconds() < 1:
        print("find_line_2")
        move_after_turning(speed, 0, 3)
        find_the_finish_line()


def dance_360(turn, times):
    if turn == 'left':
        for _ in range(times):
            zumi.reset_gyro()
            zumi.signal_left_on()
            angle_to_turn = 360
            while angle_to_turn > 10:
                zumi.turn_left(angle_to_turn)
                angle_to_turn = 360 - zumi.read_z_angle()
            zumi.signal_left_off()
            zumi.stop()
        return angle_to_turn
    elif turn == 'right':
        for _ in range(times):
            zumi.reset_gyro()
            zumi.signal_right_on()
            angle_to_turn = 360
            while angle_to_turn > 10:
                zumi.turn_right(angle_to_turn)
                angle_to_turn = 360 + zumi.read_z_angle()
            zumi.signal_right_off()
            zumi.stop()
        return angle_to_turn

def what_after_object(speed, number_of_objects, number_of_faces, threshold):
    number_of_turns = number_of_objects - number_of_faces
    camera.start_camera()
    for _ in range(5):
        frame = camera.capture()
        camera.show_image(frame)
        qr_code = vision.find_QR_code(frame)
        message = vision.get_QR_message(qr_code)
        if message != None:
            camera.close()
            screen.draw_text_center("QR Code Detected!")
            log_event("qr_code_command:"+ str(message))
            print("QR code detected!")
            qr_code_command(message, speed, number_of_turns , threshold)
            log_event("qr_code_command:"+ str(message) +"done")
            break
        gray_picture = vision.convert_to_gray(frame)
        faces= vision.find_face(gray_picture)
        if faces != None:
            camera.close()
            screen.draw_text_center("Face Detected!")
            log_event("face_detected")
            print("Face detected!")
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = str(timestamp)+".png"
            cv2.imwrite(filename, gray_picture)
            number_of_faces += 1
            break
        time.sleep(1)
    camera.close()
    return number_of_faces

def qr_code_command(message, speed, number_of_turns, threshold):
    print("QR code message: ", message)
    if message == "Left Circle":
        circle('left', number_of_turns, threshold)
    elif message == "Right Circle":
        circle('right', number_of_turns, threshold)
    elif message == "Turn Left":
        zumi.signal_left_on()
        zumi.turn_left(90)
        zumi.signal_left_off()
    elif message == "Turn Right":
        zumi.signal_right_on()
        zumi.turn_right(90)
        zumi.signal_right_off()
    elif message == "Stop":
        zumi.stop()
    elif message == "Zumi is happy today!":
        personality.happy()
    elif message == "Zumi is angry today!":
        personality.angry()
    elif message == "Zumi is celebrating today!":
        personality.celebrate()
    elif "360" in str(message):

        spin_message = message.split(" ")
        times = int(spin_message[0])
        turn = spin_message[3][:-1]
        emotion = spin_message[-1]

        print("turn", turn)
        angle_to_turn = dance_360(turn, times)*2

        if emotion == "happy":
            personality.happy()
        elif emotion == "angry":
            personality.angry()
        elif emotion == "celebrating":
            personality.celebrate()
        zumi.stop()
        
        print("Angle to turn: ", angle_to_turn)
        find_line(angle_to_turn)
    else:
        print("Invalid command")
        return "Invalid command"


def finish_with_180_turn():
    zumi.stop()
    log_event("finish")
    print("Reached end. Performing 180° turn.")
    screen.draw_text_center("Finisher box\nTurning 180°")
    zumi.turn_left(180)
    screen.draw_text_center("Done!")

def save_dict_to_csv(data_dict):
    # Generate file name with current time
    current_time = datetime.now().strftime('%Y%m%d_%H%M%S')
    file_name = "Zumi7337_output_" + current_time + ".csv"

    # Create empty list to store rows
    rows = []

    # Go through all actions and timestamps
    for action, timestamps in data_dict.items():
        for timestamp in timestamps:
            # Add row to list of rows
            rows.append({"timestamp": timestamp, "action": action})

    # Create DataFrame from list of rows
    df = pd.DataFrame(rows)

    # Sort DataFrame by column timestamp
    df = df.sort_values(by='timestamp')

    # Save DataFrame in CSV file
    df.to_csv(file_name, index=False)
    print("Data saved in ", file_name)

In [60]:
log = {}
zumi.reset_gyro()
desired_angle = zumi.read_z_angle() 
number_of_objects = 0
number_of_faces = 0

log_event('start')
log_event('end_line')
time.sleep(1)

try:
    while True:
        # Set the threshold for the IR sensors and the speed
        threshold = 50 
        speed = 5

        if object_detected():
            zumi.stop()

            zumi.play_note(1, 500) # 1 is note type (1 - 60), 500 is duration in ms
            log_event('object_detected')
            screen.draw_text_center("Object detected") #Message that object was detected
            print("Waiting for object to be removed...")
            
            zumi.brake_lights_on()
            while object_detected():
                zumi.stop()
                time.sleep(0.1)
            zumi.brake_lights_off()
            
            number_of_objects += 1

            log_event('object_removed')
            print("Object removed. Resuming movement.")
            log_event('loking for qr_code of face')
            
            number_of_faces = what_after_object(speed, number_of_objects, number_of_faces, threshold)
        if any("Zumi is" in key for key in log):
            zumi.stop()
            log_event('stop')
            finish_with_180_turn()
            log_event('finish_with_180_turn')
            break
            

        # Read all IR sensor values
        front_right, bottom_right, back_right, bottom_left, back_left, front_left = zumi.get_all_IR_data()
        
        # Correction to line if one sensor is on the line and the other is off
        desired_angle = line_correction(bottom_left, bottom_right, desired_angle, threshold)
        time.sleep(0.01) 
        print("desired_angle:", desired_angle)
        # Move forward with the corrected heading
        if bottom_left > threshold or bottom_right > threshold:
            zumi.go_straight(speed, desired_angle)
        else:
            log_event('end_line')
            if "done right circle" in log:
                if (log['done right circle'][-1]- datetime.now()).total_seconds() < 3:
                    zumi.stop()
                    time.sleep(3)
                    log['finish right circle'] = log.pop('done right circle')
            elif "done left circle" in log:
                if(log['done left circle'][-1]- datetime.now()).total_seconds() < 3:  
                    zumi.stop()
                    time.sleep(3) 
                    log['finish left circle'] = log.pop('done left circle')
            if (log['end_line'][-1] - log['end_line'][-2]).total_seconds() > 3:
                go_left = True
            
                log_event('check_left')
                # Turn to check if left is line
                turned_left_angle = turn_to_check('left')

                # Calculate angle if turn was too much or not enough
                desired_angle = turning_correction(turned_left_angle, 90)

                front_right, bottom_right, back_right, bottom_left, back_left, front_left = zumi.get_all_IR_data()
            else:
                go_left = False
            if (bottom_left > threshold or bottom_right > threshold) and go_left:
                log_event('move_left')
                move_after_turning(speed, desired_angle)
            else:
                
                log_event('check_right')
                # Turn to check if right is line
                turned_right_angle = turn_to_check('right') 
                
                # Calculate angle if turn was too much or not enough
                desired_angle = turning_correction(turned_right_angle, 180)

                front_right, bottom_right, back_right, bottom_left, back_left, front_left = zumi.get_all_IR_data()

                if bottom_left > threshold or bottom_right > threshold:
                    log_event('move_right')
                    move_after_turning(speed, desired_angle)
                else:
                    desired_angle = turn_to_check('left')
                    while (bottom_left < threshold or bottom_right < threshold):
                        front_right, bottom_right, back_right, bottom_left, back_left, front_left = zumi.get_all_IR_data()
                        zumi.go_reverse(speed, desired_angle)

finally:
    zumi.stop()
    log_event('stop')
    #save_dict_to_csv(log)

In [61]:
save_dict_to_csv(log)